In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

#my imports
import env
import os
import wrangle as w
import explore as e

In [2]:
#Get my data
df = w.acquire_zillow()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      52441 non-null  int64  
 1   propertylandusetypeid         52441 non-null  float64
 2   typeconstructiontypeid        76 non-null     float64
 3   storytypeid                   47 non-null     float64
 4   airconditioningtypeid         13638 non-null  float64
 5   architecturalstyletypeid      70 non-null     float64
 6   buildingclasstypeid           0 non-null      float64
 7   heatingorsystemtypeid         33935 non-null  float64
 8   id                            52441 non-null  int64  
 9   basementsqft                  47 non-null     float64
 10  bathroomcnt                   52441 non-null  float64
 11  bedroomcnt                    52441 non-null  float64
 12  buildingqualitytypeid         33740 non-null  float64
 13  c

In [4]:
df = w.prepare_zillow(df)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50709 entries, 0 to 52440
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   parcel_id                 50709 non-null  int64  
 1   propertylandusetypeid     50709 non-null  float64
 2   basement_sqft             50709 non-null  float64
 3   bath_count                50709 non-null  float64
 4   bed_count                 50709 non-null  float64
 5   calc_bath_and_bed         50709 non-null  float64
 6   decktypeid                50709 non-null  float64
 7   calc_sqft                 50709 non-null  float64
 8   finished_sqft12           50709 non-null  float64
 9   fips                      50709 non-null  float64
 10  fireplace_cnt             50709 non-null  float64
 11  full_bath_cnt             50709 non-null  float64
 12  has_hot_tub               50709 non-null  float64
 13  latitude                  50709 non-null  float64
 14  longit

In [ ]:
#Ensuring no nulls are present
df.isna().sum().sum()

In [ ]:
df.head().T

In [ ]:
#Drop non-numeric columns not needed for exploration:
df.drop(['taxdelinquencyflag', 'transactiondate', 'typeconstructiondesc', 'propertylandusedesc'], axis=1, inplace=True)

In [ ]:
df.head().T

In [ ]:
#Renaming columns to something readable: shoutout to Woody
rename_dict = {
    'parcelid':'parcel_id', 'basementsqft':'basement_sqft',
    'bathroomcnt':'bath_count', 'bedroomcnt':'bed_count',
       'calculatedbathnbr':'calc_bath_and_bed','finishedfloor1squarefeet':'finished_floor1_sqft',
       'calculatedfinishedsquarefeet':'calc_sqft', 'finishedsquarefeet12':'finished_sqft12',
       'finishedsquarefeet13':'finished_sqft13', 'finishedsquarefeet15':'finished_sqft15',
        'finishedsquarefeet50':'finished_sqft50',
       'finishedsquarefeet6':'finished_sqft6', 'fireplacecnt':'fireplace_cnt',
        'fullbathcnt':'full_bath_cnt',
       'garagecarcnt':'garage_car_count', 'garagetotalsqft':'garage_sqft',
       'hashottuborspa':'has_hot_tub',
        'lotsizesquarefeet':'lot_sqft', 'poolcnt':'pool_count', 'poolsizesum':'sum_pool_size',
        'propertycountylandusecode':'property_county_use_code',
        'propertyzoningdesc':'property_zoning_desc',
       'rawcensustractandblock':'raw_census_tract_block', 'regionidcity':'region_id_city',
        'regionidcounty':'region_id_county',
       'regionidneighborhood':'region_id_neighbor', 'regionidzip':'region_id_zip',
        'roomcnt':'room_count', 'threequarterbathnbr':'three_quarter_bath',
       'unitcnt':'unit_count', 'yardbuildingsqft17':'yard_building_sqft17',
        'yardbuildingsqft26':'yard_building_sqft26', 'yearbuilt':'year_built',
       'numberofstories':'no_stories', 'fireplaceflag':'fireplace_flag',
        'structuretaxvaluedollarcnt':'structure_tax_value',
       'taxvaluedollarcnt':'tax_value', 'assessmentyear':'assessment_year',
        'landtaxvaluedollarcnt':'land_value',
       'taxamount':'tax_amount', 'taxdelinquencyflag':'tax_delinquency_flag',
       'taxdelinquencyyear':'tax_delinquency_year',
       'censustractandblock':'census_tract_and_block', 'logerror':'log_error',
       'transactiondate':'transaction_date',
       'airconditioningdesc':'air_conditioning_desc',
       'architecturalstyledesc':'architectural_style_desc',
       'buildingclassdesc':'building_class_desc',
       'heatingorsystemdesc':'heating_system_desc', 'propertylandusedesc':'property_land_use_desc',
        'storydesc':'story_desc',
       'typeconstructiondesc':'type_construction_desc'
}

In [ ]:
df.rename(columns=rename_dict, inplace=True)

In [ ]:
df.head().T

In [ ]:
def get_upper_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, .75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

def add_upper_outlier_columns(df, k):
    '''
    Add a column with the suffix _outliers for all the numeric columns
    in the given dataframe.
    '''
    # outlier_cols = {col + '_outliers': get_upper_outliers(df[col], k)
    #                 for col in df.select_dtypes('number')}
    # return df.assign(**outlier_cols)

    for col in df.select_dtypes('number'):
        df[col + '_outliers'] = get_upper_outliers(df[col], k)

    return df

add_upper_outlier_columns(df, k=1.5)

df.head()

In [ ]:
outlier_cols = [col for col in df if col.endswith('_outliers')]
for col in outlier_cols:
    print('~~~\n' + col)
    data = df[col][df[col] > 0]
    print(data.describe())

In [ ]:
def create_features(df):
    df['age'] = 2017 - df.year_built
    df['age_bin'] = pd.cut(df.age, 
                           bins = [0, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140],
                           labels = [0, .066, .133, .20, .266, .333, .40, .466, .533, 
                                     .60, .666, .733, .8, .866, .933])

    # create taxrate variable
    df['taxrate'] = df.tax_amount/df.tax_value*100

    # create acres variable
    df['acres'] = df.lot_sqft/43560

    # bin acres
    df['acres_bin'] = pd.cut(df.acres, bins = [0, .10, .15, .25, .5, 1, 5, 10, 20, 50, 200], 
                       labels = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9])

    # square feet bin
    df['sqft_bin'] = pd.cut(df.calc_sqft, 
                            bins = [0, 800, 1000, 1250, 1500, 2000, 2500, 3000, 4000, 7000, 12000],
                            labels = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9]
                       )

    # dollar per square foot-structure
    df['structure_dollar_per_sqft'] = df.structure_tax_value/df.calc_sqft


    df['structure_dollar_sqft_bin'] = pd.cut(df.structure_dollar_per_sqft, 
                                             bins = [0, 25, 50, 75, 100, 150, 200, 300, 500, 1000, 1500],
                                             labels = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9]
                                            )


    # dollar per square foot-land
    df['land_dollar_per_sqft'] = df.land_value/df.lot_sqft

    df['lot_dollar_sqft_bin'] = pd.cut(df.land_dollar_per_sqft, bins = [0, 1, 5, 20, 50, 100, 250, 500, 1000, 1500, 2000],
                                       labels = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9]
                                      )


    # update datatypes of binned values to be float
    df = df.astype({'sqft_bin': 'float64', 'acres_bin': 'float64', 'age_bin': 'float64',
                    'structure_dollar_sqft_bin': 'float64', 'lot_dollar_sqft_bin': 'float64'})


    # ratio of bathrooms to bedrooms
    df['bath_bed_ratio'] = df.bath_count/df.bed_count

    # 12447 is the ID for city of LA. 
    # I confirmed through sampling and plotting, as well as looking up a few addresses.
    df['cola'] = df['region_id_city'].apply(lambda x: 1 if x == 12447.0 else 0)

    return df

In [ ]:
df = e.create_features(df)

In [ ]:
df.head().T